# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Kaggle Assignment: **

**Student Name: Jason Walker**

# Overview

## Description
This is one of the projects from the course T81-855: Applications of Deep Learning at Washington University in St. Louis. All students must create a Kaggle account and submit a solution. Once you have submitted your solution entry log into Blackboard (at WUSTL) and submit a single file telling me your Kaggle name on the leaderboard (you do not need to register to Kaggle with your real name). This competition will be visible to the public, so there may be non-student submissions as well as student.

The data set for this competition consists of a number of input columns that should be used to predict a stores sales. This is a regression problem. The inputs are a mixture of discrete and category values. The data set is from a simulation.

## Evaluation

The evaluation pages describes how submissions will be scored and how students should format their submissions. The scores are in RMSE.
Submission Format

For every store in the dataset, submission files should contain a sales volume.

The file should contain a header and have the following format:

```
100000,1.23
100001,1.123
100002,3.332
100003,1.53
...
```

# Data

## Data Description

The data contains data and costs for various office supplies. The data came from a simulation and do not directly correspond to any real-world items. See how well you can predict the cost of an item using the provided data. Feature engineering will likely help you. The *name* column may seem useless at first glance; however, it contains information that you can parse to help your predictions.
File descriptions
```
    id - The identifier/primary key.
    name - The name of this item.
    manufacturer - The manufacturer.
    pack - The number of items in this pack.
    weight - The weight of a pack of these items.
    height - The height of a pack of these items.
    width - The width of a pack of these items.
    length - The length of a pack of these items.
    cost - The cost for this item pack. This is what you are to predict (the target). 
```

# Helpful Functions

You will see these at the top of every module and assignment.  These are simply a set of reusable functions that we will make use of.  Each of them will be explained as the semester progresses.  They are explained in greater detail as the course progresses.  Class 4 contains a complete overview of these functions.

In [1]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import requests
import base64


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low
        
# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Kaggle Code



In [2]:
import os
import re
import sklearn

import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn import metrics
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split, KFold
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import regularizers

path = './data'

filename_test = os.path.join(path,"test.csv")
filename_train = os.path.join(path,"train.csv")
filename_sample = os.path.join(path,"sample.csv")
filename_submit = os.path.join(path,"submit.csv")
filename_checkpoint = os.path.join(path,"checkpoint.hdf5")

df_train = pd.read_csv(filename_train,na_values=['NA','?'])

np.random.seed(42) # Uncomment this line to get the same shuffle each time
df_train = df_train.reindex(np.random.permutation(df_train.index))
df_train.reset_index(inplace=True, drop=True)

# Encode Features

# Remove all rows where the specified column is +/- sd standard deviations
def extract_and_encode_features(df):
    color_regex='(?P<color>red|blue|green|yellow|orange|pink|black|brown|white)'
    df['color'] = df.name.str.extract(color_regex, flags=re.IGNORECASE, expand=False)

    quality_regex='(?P<quality>generic|high\squality)'
    df['quality'] = df.name.str.extract(quality_regex, flags=re.IGNORECASE, expand=False)

    size_regex='(?P<size>tiny|small|medium|large)'
    df['size'] = df.name.str.extract(size_regex, flags=re.IGNORECASE, expand=False)

    item_regex='(?P<item>paperclips|paperweights|ink\spens|pencils|stapler|tablets|thumbtacks|post\sit\snotes)'
    df['item'] = df.name.str.extract(item_regex, flags=re.IGNORECASE, expand=False)
    
    for column in ['pack','weight','height','width','length']:
        missing_median(df,column)
    
    #df.insert(1,'surface_area',(df['height']*df['width']*df['length']).astype(int))
    
    ## encode numeric features
    #for column in ['pack','weight','height','width','length','surface_area']:
    #    encode_numeric_zscore(df,column)

    # encode text/categorical features
    for column in ['manufacturer','color','quality','size','item']:
        encode_text_dummy(df,column)
  
extract_and_encode_features(df_train)

ids_train = df_train['id']
df_train.drop('id',1,inplace=True)

names_train = df_train['name']
df_train.drop('name',1,inplace=True)

x,y = to_xy(df_train,'cost')

# Used before KFold
#x_train, x_test, y_train, y_test = train_test_split(    
#    x, y, test_size=0.25, random_state=45)


/Users/jwalker/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [ ]:
# Simple function to evaluate the coefficients of a regression
%matplotlib inline    
from IPython.display import display, HTML    

def report_coef(names,coef,intercept):
    r = pd.DataFrame( { 'coef': coef, 'positive': coef>=0  }, index = names )
    r = r.sort_values(by=['coef'])
    display(r)
    print("Intercept: {}".format(intercept))
    r['coef'].plot(kind='barh', color=r['positive'].map({True: 'b', False: 'r'}))
    
# Create linear regression
regressor = sklearn.linear_model.LinearRegression()

# Fit/train linear regression
regressor.fit(x_train,y_train)
# Predict
pred = regressor.predict(x_test)

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

names = list(df_train.columns.values)
names.remove("cost")
print(names)
report_coef(
  names,
  regressor.coef_[0,:],
  regressor.intercept_)

In [ ]:


import sklearn
from sklearn.linear_model import Lasso

# Create linear regression
regressor = Lasso(random_state=0,alpha=0.01)

# Fit/train LASSO
regressor.fit(x_train,y_train)
# Predict
pred = regressor.predict(x_test)

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

names = list(df_train.columns.values)
names.remove("cost")
report_coef(
  names,
  regressor.coef_,
  regressor.intercept_)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

lasso = Lasso(random_state=42)
alphas = np.logspace(-8, 8, 10)

scores = list()
scores_std = list()

n_folds = 3

for alpha in alphas:
    lasso.alpha = alpha
    this_scores = cross_val_score(lasso, x, y, cv=n_folds, n_jobs=1)
    scores.append(np.mean(this_scores))
    scores_std.append(np.std(this_scores))

scores, scores_std = np.array(scores), np.array(scores_std)

plt.figure().set_size_inches(8, 6)
plt.semilogx(alphas, scores)

# plot error lines showing +/- std. errors of the scores
std_error = scores_std / np.sqrt(n_folds)

plt.semilogx(alphas, scores + std_error, 'b--')
plt.semilogx(alphas, scores - std_error, 'b--')

# alpha=0.2 controls the translucency of the fill color
plt.fill_between(alphas, scores + std_error, scores - std_error, alpha=0.2)

plt.ylabel('CV score +/- std error')
plt.xlabel('alpha')
plt.axhline(np.max(scores), linestyle='--', color='.5')
plt.xlim([alphas[0], alphas[-1]])

In [ ]:
import sklearn
from sklearn.linear_model import Ridge

# Create linear regression
regressor = Ridge(alpha=1)

# Fit/train Ridge
regressor.fit(x_train,y_train)
# Predict
pred = regressor.predict(x_test)

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

names = list(df_train.columns.values)
names.remove("cost")

report_coef(
  names,
  regressor.coef_[0,:],
  regressor.intercept_)

In [ ]:
import sklearn
from sklearn.linear_model import ElasticNet

# Create linear regression
regressor = ElasticNet(alpha=0.01, l1_ratio=0.1)

# Fit/train LASSO
regressor.fit(x_train,y_train)
# Predict
pred = regressor.predict(x_test)

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

names = list(df_train.columns.values)
names.remove("cost")

report_coef(
  names,
  regressor.coef_,
  regressor.intercept_)

In [3]:
# Cross-Validate
kf = KFold(5)

checkpoint = ModelCheckpoint(filepath=filename_checkpoint, verbose=0, save_best_only=True)

# Turn off KFold
#if (0):
oos_y = []
oos_pred = []
fold = 0
    
for train, test in kf.split(x):
    fold+=1
    print("Fold #{}".format(fold))
        
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]
    
    model = Sequential()
    model.add(Dense(100, input_dim=x.shape[1], activation='relu')) # Hidden 1
    model.add(Dropout(0.01)) # Dropout Layer
    model.add(Dense(50, activation='relu')) # Hidden 2
    model.add(Dense(25, 
                    kernel_regularizer=regularizers.l2(0.01), #L2 regularization
                    #activity_regularizer=regularizers.l1(0.01), #L1 Lasso regularization
                    activation='relu')) # Hidden 3 
    model.add(Dense(10, activation='relu')) # Hidden 4
    model.add(Dense(1)) # Output
    model.compile(loss='mean_squared_error', optimizer='adam')
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=1000, verbose=1, mode='auto')
    model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpoint],verbose=1,epochs=10000)
    
    pred = model.predict(x_test)
    
    oos_y.append(y_test)
    oos_pred.append(pred)        

    # Measure this fold's RMSE
    score = np.sqrt(metrics.mean_squared_error(pred,y_test))
    print("Fold score (RMSE): {}".format(score))


Fold #1
Train on 8000 samples, validate on 2000 samples
Epoch 1/10000
8000/8000 [==============================] - 1s 134us/step - loss: 5184.6087 - val_loss: 4263.2856
Epoch 2/10000
8000/8000 [==============================] - 0s 62us/step - loss: 4795.4176 - val_loss: 3766.8765
Epoch 3/10000
8000/8000 [==============================] - 1s 77us/step - loss: 4533.3148 - val_loss: 3618.5896
Epoch 4/10000
8000/8000 [==============================] - 0s 62us/step - loss: 4330.8708 - val_loss: 3535.3761
Epoch 5/10000
8000/8000 [==============================] - 1s 64us/step - loss: 4197.3030 - val_loss: 3501.0222
Epoch 6/10000
8000/8000 [==============================] - 1s 64us/step - loss: 4167.7544 - val_loss: 3478.9662
Epoch 7/10000
8000/8000 [==============================] - 0s 61us/step - loss: 4200.6497 - val_loss: 3384.7174
Epoch 8/10000
8000/8000 [==============================] - 0s 62us/step - loss: 4018.0491 - val_loss: 3342.5486
Epoch 9/10000
8000/8000 [======================

Epoch 74/10000
8000/8000 [==============================] - 0s 51us/step - loss: 317.5942 - val_loss: 315.5768
Epoch 75/10000
8000/8000 [==============================] - 0s 52us/step - loss: 346.5683 - val_loss: 233.6568
Epoch 76/10000
8000/8000 [==============================] - 0s 50us/step - loss: 290.1542 - val_loss: 206.6138
Epoch 77/10000
8000/8000 [==============================] - 0s 52us/step - loss: 361.1052 - val_loss: 418.9859
Epoch 78/10000
8000/8000 [==============================] - 0s 50us/step - loss: 279.6595 - val_loss: 535.5874
Epoch 79/10000
8000/8000 [==============================] - 0s 49us/step - loss: 281.8755 - val_loss: 246.2134
Epoch 80/10000
8000/8000 [==============================] - 0s 51us/step - loss: 296.4417 - val_loss: 491.6167
Epoch 81/10000
8000/8000 [==============================] - 0s 52us/step - loss: 295.6843 - val_loss: 195.3086
Epoch 82/10000
8000/8000 [==============================] - 0s 51us/step - loss: 308.0164 - val_loss: 360.1060
E

8000/8000 [==============================] - 0s 51us/step - loss: 226.4310 - val_loss: 170.2758
Epoch 148/10000
8000/8000 [==============================] - 0s 50us/step - loss: 222.3887 - val_loss: 275.6519
Epoch 149/10000
8000/8000 [==============================] - 0s 50us/step - loss: 239.9952 - val_loss: 336.6002
Epoch 150/10000
8000/8000 [==============================] - 0s 50us/step - loss: 237.9033 - val_loss: 210.4989
Epoch 151/10000
8000/8000 [==============================] - 0s 48us/step - loss: 356.4307 - val_loss: 202.7259
Epoch 152/10000
8000/8000 [==============================] - 0s 51us/step - loss: 204.4258 - val_loss: 273.0073
Epoch 153/10000
8000/8000 [==============================] - 0s 50us/step - loss: 244.1685 - val_loss: 216.6297
Epoch 154/10000
8000/8000 [==============================] - 0s 51us/step - loss: 229.8870 - val_loss: 233.2082
Epoch 155/10000
8000/8000 [==============================] - 0s 56us/step - loss: 337.5540 - val_loss: 255.7913
Epoch 15

8000/8000 [==============================] - 0s 49us/step - loss: 219.3102 - val_loss: 366.6412
Epoch 221/10000
8000/8000 [==============================] - 0s 49us/step - loss: 195.0191 - val_loss: 195.4993
Epoch 222/10000
8000/8000 [==============================] - 0s 50us/step - loss: 177.9925 - val_loss: 171.6863
Epoch 223/10000
8000/8000 [==============================] - 0s 48us/step - loss: 203.4616 - val_loss: 179.3473
Epoch 224/10000
8000/8000 [==============================] - 0s 50us/step - loss: 169.6172 - val_loss: 151.2030
Epoch 225/10000
8000/8000 [==============================] - 0s 52us/step - loss: 184.6813 - val_loss: 149.3017
Epoch 226/10000
8000/8000 [==============================] - 0s 49us/step - loss: 199.8051 - val_loss: 183.5751
Epoch 227/10000
8000/8000 [==============================] - 0s 48us/step - loss: 199.0026 - val_loss: 166.6880
Epoch 228/10000
8000/8000 [==============================] - 0s 57us/step - loss: 193.8336 - val_loss: 233.1144
Epoch 22

8000/8000 [==============================] - 0s 48us/step - loss: 166.3002 - val_loss: 164.3069
Epoch 294/10000
8000/8000 [==============================] - 0s 49us/step - loss: 161.7628 - val_loss: 161.0776
Epoch 295/10000
8000/8000 [==============================] - 0s 48us/step - loss: 172.7813 - val_loss: 187.2861
Epoch 296/10000
8000/8000 [==============================] - 0s 49us/step - loss: 272.7146 - val_loss: 255.3957
Epoch 297/10000
8000/8000 [==============================] - 0s 49us/step - loss: 187.0678 - val_loss: 149.0603
Epoch 298/10000
8000/8000 [==============================] - 0s 50us/step - loss: 457.5000 - val_loss: 208.6890
Epoch 299/10000
8000/8000 [==============================] - 0s 50us/step - loss: 157.6518 - val_loss: 151.6693
Epoch 300/10000
8000/8000 [==============================] - 0s 52us/step - loss: 307.2444 - val_loss: 714.0228
Epoch 301/10000
8000/8000 [==============================] - 0s 51us/step - loss: 278.1326 - val_loss: 159.4155
Epoch 30

...

8000/8000 [==============================] - 0s 53us/step - loss: 117.2446 - val_loss: 111.1630
Epoch 3168/10000
8000/8000 [==============================] - 0s 54us/step - loss: 115.1278 - val_loss: 109.1392
Epoch 3169/10000
8000/8000 [==============================] - 0s 53us/step - loss: 123.9319 - val_loss: 120.1794
Epoch 3170/10000
8000/8000 [==============================] - 0s 51us/step - loss: 119.1298 - val_loss: 119.2128
Epoch 3171/10000
8000/8000 [==============================] - 0s 54us/step - loss: 126.5793 - val_loss: 115.8182
Epoch 3172/10000
8000/8000 [==============================] - 0s 54us/step - loss: 120.0016 - val_loss: 137.3890
Epoch 3173/10000
8000/8000 [==============================] - 0s 53us/step - loss: 115.6308 - val_loss: 123.0854
Epoch 3174/10000
8000/8000 [==============================] - 0s 57us/step - loss: 133.4137 - val_loss: 128.4945
Epoch 3175/10000
8000/8000 [==============================] - 0s 53us/step - loss: 121.6466 - val_loss: 116.1437


8000/8000 [==============================] - 0s 52us/step - loss: 120.5036 - val_loss: 114.6237
Epoch 3240/10000
8000/8000 [==============================] - 0s 54us/step - loss: 118.1057 - val_loss: 126.7445
Epoch 3241/10000
8000/8000 [==============================] - 0s 53us/step - loss: 116.3357 - val_loss: 124.7263
Epoch 3242/10000
8000/8000 [==============================] - 0s 57us/step - loss: 149.2005 - val_loss: 137.3089
Epoch 3243/10000
8000/8000 [==============================] - 0s 61us/step - loss: 127.0295 - val_loss: 119.3839
Epoch 3244/10000
8000/8000 [==============================] - 0s 56us/step - loss: 114.8968 - val_loss: 134.4037
Epoch 3245/10000
8000/8000 [==============================] - 0s 61us/step - loss: 114.1869 - val_loss: 119.6807
Epoch 3246/10000
8000/8000 [==============================] - 0s 55us/step - loss: 112.0394 - val_loss: 122.8925
Epoch 3247/10000
8000/8000 [==============================] - 1s 71us/step - loss: 122.8733 - val_loss: 113.9581


8000/8000 [==============================] - 0s 56us/step - loss: 119.9338 - val_loss: 125.3099
Epoch 3312/10000
8000/8000 [==============================] - 0s 55us/step - loss: 118.9930 - val_loss: 114.8201
Epoch 3313/10000
8000/8000 [==============================] - 0s 53us/step - loss: 110.1554 - val_loss: 119.1134
Epoch 3314/10000
8000/8000 [==============================] - 0s 54us/step - loss: 113.5843 - val_loss: 106.2401
Epoch 3315/10000
8000/8000 [==============================] - 1s 68us/step - loss: 120.0634 - val_loss: 140.1558
Epoch 3316/10000
8000/8000 [==============================] - 1s 88us/step - loss: 125.4349 - val_loss: 123.1548
Epoch 3317/10000
8000/8000 [==============================] - 1s 75us/step - loss: 114.8204 - val_loss: 120.7070
Epoch 3318/10000
8000/8000 [==============================] - 0s 61us/step - loss: 125.3860 - val_loss: 116.4724
Epoch 3319/10000
8000/8000 [==============================] - 0s 57us/step - loss: 120.4488 - val_loss: 141.5382


8000/8000 [==============================] - 0s 55us/step - loss: 117.1542 - val_loss: 153.5233
Epoch 3384/10000
8000/8000 [==============================] - 0s 52us/step - loss: 124.5667 - val_loss: 216.9253
Epoch 3385/10000
8000/8000 [==============================] - 0s 54us/step - loss: 114.9340 - val_loss: 118.7555
Epoch 3386/10000
8000/8000 [==============================] - 0s 54us/step - loss: 115.7449 - val_loss: 135.5622
Epoch 3387/10000
8000/8000 [==============================] - 0s 56us/step - loss: 126.4091 - val_loss: 141.3227
Epoch 3388/10000
8000/8000 [==============================] - 0s 56us/step - loss: 125.8545 - val_loss: 121.0311
Epoch 3389/10000
8000/8000 [==============================] - 0s 56us/step - loss: 124.6718 - val_loss: 131.6303
Epoch 3390/10000
8000/8000 [==============================] - 0s 59us/step - loss: 126.7409 - val_loss: 115.5611
Epoch 3391/10000
8000/8000 [==============================] - 0s 57us/step - loss: 118.1754 - val_loss: 113.5832


8000/8000 [==============================] - 0s 53us/step - loss: 117.4476 - val_loss: 115.6148
Epoch 3456/10000
8000/8000 [==============================] - 0s 54us/step - loss: 132.7667 - val_loss: 121.1742
Epoch 3457/10000
8000/8000 [==============================] - 1s 64us/step - loss: 121.5520 - val_loss: 121.5298
Epoch 3458/10000
8000/8000 [==============================] - 1s 66us/step - loss: 116.3358 - val_loss: 111.8179
Epoch 3459/10000
8000/8000 [==============================] - 0s 57us/step - loss: 115.6753 - val_loss: 151.4354
Epoch 3460/10000
8000/8000 [==============================] - 0s 54us/step - loss: 124.4842 - val_loss: 110.4219
Epoch 3461/10000
8000/8000 [==============================] - 0s 58us/step - loss: 115.3715 - val_loss: 168.6867
Epoch 3462/10000
8000/8000 [==============================] - 0s 57us/step - loss: 127.5873 - val_loss: 114.4067
Epoch 3463/10000
8000/8000 [==============================] - 0s 54us/step - loss: 116.9139 - val_loss: 126.5519


In [ ]:
# Build the oos prediction list and calculate the error.
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
score = np.sqrt(metrics.mean_squared_error(oos_pred,oos_y))
print("Final, out of sample score (RMSE): {}".format(score)) 

In [ ]:

#pred = model.predict(x_test)

# Measure RMSE error.  RMSE is common for regression.
#score = np.sqrt(metrics.mean_squared_error(pred,y_test))
#print("Final score (RMSE): {}".format(score))

#chart_regression(pred.flatten(),y_test)
#chart_regression(pred.flatten(),y_test,sort=False)

In [ ]:
model.load_weights(filename_checkpoint)

df_test = pd.read_csv(filename_test,na_values=['NA','?'])

extract_and_encode_features(df_test)

ids_test = df_test['id']
df_test.drop('id',1,inplace=True)

names_test = df_test['name']
df_test.drop('name',1,inplace=True)

x_submit = df_test.as_matrix().astype(np.float32)
pred_submit = model.predict(x_submit)


In [ ]:
cost = [n if n > 0 else n * -1 for n in pred_submit[:,0]]
df_submit = pd.DataFrame({'id': ids_test,'cost': cost})
df_submit = df_submit[['id', 'cost']]
df_submit.to_csv(filename_submit, index=False)